In [ ]:
import pandas as pd
import numpy as np

ems_logs = pd.read_csv("../data/1_raw_data/Fire_Department_and_Emergency_Medical_Services_Dispatched_Calls_for_Service.csv") 


keep_cols = [
    "Call Number", "Incident Number", "Unit ID",
    "Call Type Group", "Call Type", "Unit Type",
    "Dispatch DtTm", "Response DtTm", "On Scene DtTm",
    "City", "case_location", "Address", "Priority", "Original Priority"
]
keep_cols = [c for c in keep_cols if c in df.columns]
df_small = df[keep_cols].copy()

df_small["Dispatch DtTm"] = pd.to_datetime(df_small["Dispatch DtTm"], errors="coerce")

df_small = df_small[df_small["Dispatch DtTm"].notna()]

start_mmdd = (1, 1)
end_mmdd   = (11, 21)

mmdd = df_small["Dispatch DtTm"].dt.month*100 + df_small["Dispatch DtTm"].dt.day
df_small = df_small[(mmdd >= start_mmdd[0]*100 + start_mmdd[1]) &
                    (mmdd <= end_mmdd[0]*100 + end_mmdd[1])]


mask = (
    df_small["City"].str.contains("San Francisco", case=False, na=False) |
    df_small["City"].str.contains("SF", case=False, na=False) |
    df_small["City"].str.contains("SAN FRANCISCO", case=False, na=False) 
)

df_small = df_small[mask]


df_small = df_small[df_small["Unit Type"].str.contains(
    "MEDIC|AMBUL|ALS", case=False, na=False
)]

df_small = df_small[df_small["case_location"].notna()]

def parse_case_location(x):
    if pd.isna(x):
        return np.nan, np.nan
    s = str(x).strip()

    # format: "(lat, lon)"
    if s.startswith("(") and "," in s:
        s2 = s.strip("()")
        lat, lon = s2.split(",")
        return float(lat), float(lon)

    # format: "POINT (lon lat)"
    if "POINT" in s.upper():
        s2 = s.upper().replace("POINT", "").strip(" ()")
        lon, lat = s2.split()
        return float(lat), float(lon)

    return np.nan, np.nan

df_small["lat"], df_small["lon"] = zip(*df_small["case_location"].map(parse_case_location))
df_small = df_small[df_small["lat"].notna() & df_small["lon"].notna()]

final_cols = [c for c in [
    "Call Number", "Incident Number", "Unit ID",
    "Dispatch DtTm", "Call Type Group", "Call Type",
    "Unit Type", "Priority", "lat", "lon"
] if c in df_small.columns]

ems_events = df_small[final_cols].reset_index(drop=True)

print(ems_events.head())
print("Rows kept:", len(ems_events))
ems_events.to_csv('../data/2_full_dataset/ems_logs.csv')
